In [1]:
cd D:\ADM Assignment 3\recommenders

D:\ADM Assignment 3\recommenders


In [2]:
import sys
sys.path.append("../../")
import os
import logging
import papermill as pm
from tempfile import TemporaryDirectory

import tensorflow as tf
import time

from reco_utils.common.constants import SEED
from reco_utils.recommender.deeprec.deeprec_utils import (
    prepare_hparams
)
from reco_utils.dataset.amazon_reviews import download_and_extract, data_preprocessing
from reco_utils.dataset.download_utils import maybe_download
#from reco_utils.recommender.deeprec.models.sequential.sli_rec import SLI_RECModel
####  to use the other model, use one of the following lines:
#from reco_utils.recommender.deeprec.models.sequential.asvd import A2SVDModel
#from reco_utils.recommender.deeprec.models.sequential.caser import CaserModel
from reco_utils.recommender.deeprec.models.sequential.gru4rec import GRU4RecModel
#from reco_utils.recommender.deeprec.models.sequential.nextitnet import NextItNetModel

from reco_utils.recommender.deeprec.io.sequential_iterator import SequentialIterator
#from reco_utils.recommender.deeprec.io.nextitnet_iterator import NextItNetIterator

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]
Tensorflow version: 1.15.3


In [3]:
EPOCHS = 10
BATCH_SIZE = 400
RANDOM_SEED = SEED  # Set None for non-deterministic result
yaml_file = 'D://ADM Assignment 3//recommenders//reco_utils//recommender//deeprec//config//sli_rec.yaml'
data_path = 'D://ADM Assignment 3//recommenders//tests//resources//deeprec//slirec'

In [4]:
# for test
train_file = os.path.join(data_path, r'train_data')
valid_file = os.path.join(data_path, r'valid_data')
test_file = os.path.join(data_path, r'test_data')
user_vocab = os.path.join(data_path, r'user_vocab.pkl')
item_vocab = os.path.join(data_path, r'item_vocab.pkl')
cate_vocab = os.path.join(data_path, r'category_vocab.pkl')
output_file = os.path.join(data_path, r'output.txt')

user_name = 'D://ADM Assignment 3//recommenders//Assignment3//user.csv'
snack_name = 'D://ADM Assignment 3//recommenders//Assignment3//product.csv'
reviews_file = os.path.join(data_path, snack_name)
user_file = os.path.join(data_path, user_name)
train_num_ngs = 4 # number of negative instances with a positive instance for training
valid_num_ngs = 4 # number of negative instances with a positive instance for validation
test_num_ngs = 9 # number of negative instances with a positive instance for testing
sample_rate = 0.01 # sample a small item set for training and testing here for fast example

input_files = [reviews_file, meta_file, train_file, valid_file, test_file, user_vocab, item_vocab, cate_vocab]

if not os.path.exists(train_file):
    download_and_extract(reviews_name, reviews_file)
    download_and_extract(meta_name, meta_file)
    data_preprocessing(*input_files, sample_rate=sample_rate, valid_num_ngs=valid_num_ngs, test_num_ngs=test_num_ngs)
    #### uncomment this for the NextItNet model, because it does not need to unfold the user history
    # data_preprocessing(*input_files, sample_rate=sample_rate, valid_num_ngs=valid_num_ngs, test_num_ngs=test_num_ngs, is_history_expanding=False)

In [5]:

hparams = prepare_hparams(yaml_file, 
                          embed_l2=0., 
                          layer_l2=0., 
                          learning_rate=0.001, 
                          epochs=EPOCHS,
                          batch_size=BATCH_SIZE,
                          show_step=20,
                          MODEL_DIR=os.path.join(data_path, "model/"),
                          SUMMARIES_DIR=os.path.join(data_path, "summary/"),
                          user_vocab=user_vocab,
                          item_vocab=item_vocab,
                          cate_vocab=cate_vocab,
                          need_sample=True,
                          train_num_ngs=train_num_ngs, # provides the number of negative instances for each positive instance for loss computation.
            )

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [6]:
input_creator = SequentialIterator

In [7]:
model = GRU4RecModel(hparams, input_creator, seed=RANDOM_SEED)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
P

In [8]:
print(model.run_eval(test_file, num_ngs=test_num_ngs)) # test_num_ngs is the number of negative lines after each positive line in your test_file


{'auc': 0.4979, 'logloss': 0.6931, 'mean_mrr': 0.2781, 'ndcg@2': 0.1473, 'ndcg@4': 0.2339, 'ndcg@6': 0.3075, 'group_auc': 0.4989}


In [9]:
start_time = time.time()
model = model.fit(train_file, valid_file, valid_num_ngs=valid_num_ngs) 
# valid_num_ngs is the number of negative lines after each positive line in your valid_file 
# we will evaluate the performance of model on valid_file every epoch
end_time = time.time()
print('Time cost for training is {0:.2f} mins'.format((end_time-start_time)/60.0))


step 20 , total_loss: 1.6118, data_loss: 1.6118
step 40 , total_loss: 1.6072, data_loss: 1.6072
eval valid at epoch 1: auc:0.5267,logloss:0.6932,mean_mrr:0.4771,ndcg@2:0.3548,ndcg@4:0.5368,ndcg@6:0.6055,group_auc:0.5292
step 20 , total_loss: 1.5007, data_loss: 1.5007
step 40 , total_loss: 1.4153, data_loss: 1.4153
eval valid at epoch 2: auc:0.6676,logloss:0.7269,mean_mrr:0.5864,ndcg@2:0.5095,ndcg@4:0.6581,ndcg@6:0.6894,group_auc:0.6666
step 20 , total_loss: 1.4515, data_loss: 1.4515
step 40 , total_loss: 1.3260, data_loss: 1.3260
eval valid at epoch 3: auc:0.6998,logloss:0.8063,mean_mrr:0.6124,ndcg@2:0.5484,ndcg@4:0.6803,ndcg@6:0.7092,group_auc:0.6928
step 20 , total_loss: 1.2797, data_loss: 1.2797
step 40 , total_loss: 1.3882, data_loss: 1.3882
eval valid at epoch 4: auc:0.7082,logloss:0.6688,mean_mrr:0.623,ndcg@2:0.5607,ndcg@4:0.6877,ndcg@6:0.7171,group_auc:0.6998
step 20 , total_loss: 1.3661, data_loss: 1.3661
step 40 , total_loss: 1.2706, data_loss: 1.2706
eval valid at epoch 5: a

In [10]:
res_syn = model.run_eval(test_file, num_ngs=test_num_ngs)
print(res_syn)

{'auc': 0.688, 'logloss': 0.6787, 'mean_mrr': 0.4434, 'ndcg@2': 0.3423, 'ndcg@4': 0.4568, 'ndcg@6': 0.5138, 'group_auc': 0.6794}


In [ ]:
sb.glue("res_syn", res_syn)

In [17]:
filename = 'GRU4Rec.sav'
joblib.dump(model, filename)


['GRU4Rec.sav']

In [18]:
cd

C:\Users\YashPandya


In [11]:
model_file = model.predict(test_file, output_file)
print(model_file)